In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import os

#transformations
train_transforms = transforms.Compose([transforms.Resize((224,224))])

class BrainCancerDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, transform=None):
      self.arr = []
      self.num_yes = 0
      directory = 'drive/MyDrive/brain_tumor_dataset/yes'
      for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
          self.num_yes = self.num_yes + 1
          self.arr.append(torchvision.io.read_image(f))
      directory = 'drive/MyDrive/brain_tumor_dataset/no'
      for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            self.arr.append(torchvision.io.read_image(f))
      self.transform = transform
       

    def __len__(self):
        return len(self.arr)

    def __getitem__(self, idx):
        img = self.arr[idx]
        img = self.transform(img)
        if idx >= self.num_yes:
          return img, 1
        return img, 0
train_data = BrainCancerDataset(transform=train_transforms)
#dataloader
trainloader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size=16)

In [ ]:
def make_train_step(model, optimizer, loss_fn):
  def train_step(x,y):
    #make prediction
    yhat = model(x)
    #enter train mode
    model.train()
    #compute loss
    loss = loss_fn(yhat,y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    #optimizer.cleargrads()

    return loss
  return train_step
from torchvision import datasets, models, transforms
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
model = models.resnet18(pretrained=True)

#freeze all params
for params in model.parameters():
  params.requires_grad_ = False

#add a new final layer
nr_filters = model.fc.in_features  #number of input features of last layer
model.fc = nn.Linear(nr_filters, 1)

model = model.to(device)
from torch.nn.modules.loss import BCEWithLogitsLoss
from torch.optim import lr_scheduler

#loss
loss_fn = BCEWithLogitsLoss() #binary cross entropy with sigmoid, so no need to use sigmoid in the model

#optimizer
optimizer = torch.optim.Adam(model.fc.parameters()) 

#train step
train_step = make_train_step(model, optimizer, loss_fn)

In [ ]:
from tqdm import tqdm


losses = []
val_losses = []

epoch_train_losses = []
epoch_test_losses = []

n_epochs = 10
early_stopping_tolerance = 3
early_stopping_threshold = 0.03

for epoch in range(n_epochs):
  epoch_loss = 0
  for i ,data in trainloader:
    x_batch , y_batch = data
    x_batch = x_batch.to(device) #move to gpu
    y_batch = y_batch.unsqueeze(1).float() #convert target to same nn output shape
    y_batch = y_batch.to(device) #move to gpu


    loss = train_step(x_batch, y_batch)
    epoch_loss += loss/len(trainloader)
    losses.append(loss)
    
  epoch_train_losses.append(epoch_loss)
  print('\nEpoch : {}, train loss : {}'.format(epoch+1,epoch_loss))
    

RuntimeError: ignored